# read dicom

In [10]:
import numpy as np
import skimage.measure
from skimage.transform import rescale, rotate
# import pydicom
import vtk
from vtk.util import numpy_support
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image

In [9]:
import gc
gc.collect()

44

In [4]:
# set working directory
plt.gray()
os.chdir('C:/Users/span/Documents/CNN/')

<Figure size 432x288 with 0 Axes>

In [5]:
def readdicomset(number1):
    # read dicom _dl_input
    PathDicom = "01_perprocess/"+str(number1)+'/'+str(number1)+"_5/"
    # PathDicom = "01_perprocess/"+str(number1)+'/'+str(number1)+"_in/"
    reader = vtk.vtkDICOMImageReader()
    reader.SetDirectoryName(PathDicom)
    reader.Update()
    # Load dimensions using `GetDataExtent`
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]
    # Load spacing values
    ConstPixelSpacing = reader.GetPixelSpacing()
    # Get the 'vtkImageData' object from the reader
    imageData = reader.GetOutput()
    # Get the 'vtkPointData' object from the 'vtkImageData' object
    pointData = imageData.GetPointData()
    # Ensure that only one array exists within the 'vtkPointData' object
    assert (pointData.GetNumberOfArrays()==1)
    # Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
    arrayData = pointData.GetArray(0)
    # Convert the `vtkArray` to a NumPy array
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    # Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    return ArrayDicom

In [14]:
number1 = 37
a = readdicomset(number1)
a = np.where(a <= -1000, -999, a)
a = np.moveaxis(a, 0, 1)

In [15]:
def arraytoimage(nparray, number1, a):
    image = skimage.transform.rotate(nparray,0,preserve_range=True).copy()
    I8_0 = (((image - a.min()) / (a.max() - a.min())) * 255.9)
    img0 = Image.fromarray(I8_0.astype(np.uint8))
    return img0

In [16]:
# output in folder for bmp image
pathin = "01_perprocess/"+str(number1)+'/'+str(number1)+"_in/"

for i in range(a.shape[2]):
    img0 = arraytoimage(a[:,:,i],number1,a)
    img0.save(pathin+str(number1)+"_in_rec00000"+str("{0:03}".format(i))+".bmp")

In [17]:
def arraytoimageres(nparray, number1, a, resize):
    image = skimage.transform.rotate(nparray,0,preserve_range=True).copy()
    image = rescale(image, (resize/a[:, :, 0].shape[1]), preserve_range=True, anti_aliasing=False)
    I8_0 = (((image - a.min()) / (a.max() - a.min())) * 255.9)
    img0 = Image.fromarray(I8_0.astype(np.uint8))
    return img0

In [18]:
# output prepro folder for 256 png image
path_master_i = "01_perprocess/"+str(number1)+'/'+str(number1)+"_prepro/"
for i in range(a.shape[2]):
    img0 = arraytoimageres(a[:,:,i],number1,a,256)
    img0.save(path_master_i+str(number1)+"_image_"+str("{0:03}".format(i))+".png")

# TEST ZONE

In [12]:
from scipy import ndimage as ndi

from PIL import Image

from skimage import filters, io, morphology, img_as_bool, segmentation
from skimage.filters import threshold_local
from skimage.feature import canny
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.transform import rescale, resize, downscale_local_mean

import cv2

In [7]:
# define rescale
from scipy import ndimage as ndi
from skimage import io, morphology, img_as_bool, segmentation
from skimage import filters
from skimage.transform import rescale
from skimage.measure import regionprops
from skimage import morphology
from skimage.feature import canny
import cv2

def overlay_image_center(image, rescale, block_size, manoff, imageremovesize, removesize):
#     overlay image for cut inner off
    threshold_value = skimage.filters.threshold_otsu(image)
    labeled_foreground = (image > threshold_value)
#     labeled_backeground = np.where(skimage.morphology.remove_small_objects(labeled_foreground, imageremovesize) == 0,1,0)
    center_of_mass = skimage.measure.regionprops(labeled_foreground.astype(int), labeled_foreground.astype(int))[0].centroid
    image_rescaled = skimage.transform.rescale(labeled_foreground.astype(int), rescale, anti_aliasing=False)
    image_rescaled[image_rescaled == image_rescaled.min()] = 0
    image_rescaled[image_rescaled != image_rescaled.min()] = 1
    center_of_mass1 = skimage.measure.regionprops(image_rescaled.astype(int), image_rescaled)[0].centroid
    image_rescaled3 = skimage.morphology.closing(skimage.feature.canny(image_rescaled, sigma=1))
    s_img = image_rescaled3
    l_img = labeled_foreground.astype(int)
    l_img[l_img != 0] =0
    x_offset= round(center_of_mass[1] - center_of_mass1[1]).astype(int)
    y_offset= round(center_of_mass[0] - center_of_mass1[0]).astype(int)
    l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
#     process image
    binary_image1 = image > (skimage.filters.threshold_local(image, block_size,offset=0, method='mean')+manoff)
    binary_image2 = skimage.morphology.remove_small_objects(binary_image1, imageremovesize)
#     fill_coins = ndi.binary_fill_holes(binary_image1)
    binary_image5_0 = binary_image2 > l_img
    binary_image5_0_1 = skimage.morphology.remove_small_objects(binary_image5_0, removesize)
    # background
    out = ndi.distance_transform_edt(~binary_image2)
    out = out < 0.05 * out.max()
    out = morphology.skeletonize(out)
    out = morphology.binary_dilation(out, morphology.selem.disk(1))
    out = segmentation.clear_border(out)
    out = out | binary_image2
    labeled_backeground = np.where(ndi.binary_fill_holes(out) == 0,1,0)
    binary_image5_1 = labeled_backeground + binary_image5_0_1
    binary_image5_1 = np.where(binary_image5_1 != 0,1,0)
    return binary_image5_1

In [13]:
# define rescale 2
from scipy import ndimage as ndi
from skimage import io, morphology, img_as_bool, segmentation
from skimage import filters
from skimage.transform import rescale
from skimage.measure import regionprops
from skimage import morphology
from skimage.feature import canny
import cv2

def overlay_image_center2(image, rescale, block_size, manoff, imageremovesize, removesize):
#     overlay image for cut inner off
    threshold_value = skimage.filters.threshold_otsu(image)
    labeled_foreground = (image > threshold_value)
#     labeled_backeground = np.where(skimage.morphology.remove_small_objects(labeled_foreground, imageremovesize) == 0,1,0)
    center_of_mass = skimage.measure.regionprops(labeled_foreground.astype(int), labeled_foreground.astype(int))[0].centroid
    image_rescaled = skimage.transform.rescale(labeled_foreground.astype(int), rescale, anti_aliasing=False)
    image_rescaled[image_rescaled == image_rescaled.min()] = 0
    image_rescaled[image_rescaled != image_rescaled.min()] = 1
    center_of_mass1 = skimage.measure.regionprops(image_rescaled.astype(int), image_rescaled)[0].centroid
    image_rescaled3 = skimage.morphology.closing(skimage.feature.canny(image_rescaled, sigma=1))
    s_img = image_rescaled3
    l_img = labeled_foreground.astype(int)
    l_img[l_img != 0] =0
    x_offset= round(center_of_mass[1] - center_of_mass1[1]).astype(int)
    y_offset= round(center_of_mass[0] - center_of_mass1[0]).astype(int)
    l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1]] = s_img
#     process image
    binary_image1 = image > (skimage.filters.threshold_local(image, block_size,offset=0, method='mean')+manoff)
    binary_image2 = skimage.morphology.remove_small_objects(binary_image1, imageremovesize)
#     fill_coins = ndi.binary_fill_holes(binary_image1)
    l_img1 = ndi.binary_fill_holes(l_img)
    l_img1 = np.where(l_img1 == 0,1,0)
    binary_image5_0 = binary_image1 > l_img1
#     binary_image5_0_1 = skimage.morphology.remove_small_objects(binary_image5_0, removesize)
#     # background
#     out = ndi.distance_transform_edt(~binary_image2)
#     out = out < 0.05 * out.max()
#     out = morphology.skeletonize(out)
#     out = morphology.binary_dilation(out, morphology.selem.disk(1))
#     out = segmentation.clear_border(out)
#     out = out | binary_image2
#     labeled_backeground = np.where(ndi.binary_fill_holes(out) == 0,1,0)
#     binary_image5_1 = labeled_backeground + binary_image5_0_1
#     binary_image5_1 = np.where(binary_image5_1 != 0,1,0)
    return binary_image5_0

In [8]:
from PIL import Image
pathout = "01_perprocess/"+str(number1)+'/'+str(number1)+"_out/"
result = np.empty([596, 596])
for i in range(a.shape[2]):
    image = a[:, :, i].copy()
    image[0,0] = 1
    block_size = 23
    binary_image1 = image > (threshold_local(image, block_size,offset=0, method='mean')+20)
    binary_image5_1 = overlay_image_center(image=image, 
                                       rescale=0.7, # rescale=0.6
                                       block_size=23, 
                                       manoff=20, 
                                       imageremovesize=4000, 
                                       removesize=2000)
    binary_image5 = binary_image1 > binary_image5_1
    binary_image6 = morphology.remove_small_objects(binary_image5, 50)
    I8_1 = (((binary_image6.astype(np.uint8) - binary_image6.astype(np.uint8).min()) / (binary_image6.astype(np.uint8).max() - binary_image6.astype(np.uint8).min())) * 255.9).astype(np.uint8)
    img1 = Image.fromarray(I8_1)
    img1.save(pathout+str(number1)+"_out_rec00000"+str("{0:03}".format(i))+".bmp")

C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [17]:
from PIL import Image
pathout = "01_perprocess/"+str(number1)+'/'+str(number1)+"_out2/"
result = np.empty([596, 596])
for i in range(a.shape[2]):
    image = a[:, :, i].copy()
    image[0,0] = 1
    block_size = 23
    binary_image1 = image > (threshold_local(image, block_size,offset=0, method='mean')+20)
    binary_image5_1 = overlay_image_center2(image=image, 
                                       rescale=0.7, # rescale=0.6
                                       block_size=23, 
                                       manoff=20, 
                                       imageremovesize=4000, 
                                       removesize=2000)
#     binary_image5 = binary_image1 > binary_image5_1
    binary_image6 = morphology.remove_small_objects(binary_image5_1, 50)
    I8_1 = (((binary_image6.astype(np.uint8) - binary_image6.astype(np.uint8).min()) / (binary_image6.astype(np.uint8).max() - binary_image6.astype(np.uint8).min())) * 255.9).astype(np.uint8)
    img1 = Image.fromarray(I8_1)
    img1.save(pathout+str(number1)+"_out_rec00000"+str("{0:03}".format(i))+".bmp")

C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
